In [5]:
from numpy import * 
def loadSimpData():
    dataMat = matrix([[1.0,2.1],[2.,1.1],[1.3,1.],[1.,1.],[2.,1.]])
    classLabels = [1.0,1.0,-1.0,-1.0,1.0]
    return dataMat,classLabels

def stumpClassify(dataMatrix,dimen,threshVal,threshIneq):
    retArray = ones((shape(dataMatrix)[0],1))
    if threshIneq == 'lt':
        retArray[dataMatrix[:,dimen]<=threshVal]=-1.0
    else:
        retArray[dataMatrix[:,dimen]>threshVal]=-1.0
    return retArray
def buildStump(dataArr,classLabels,D):
    dataMatrix = mat(dataArr)
    labelMat = mat(classLabels).T
    m,n = shape(dataMatrix)
    numSteps = 10.0
    bestStump = {}
    bestClassEst = mat(zeros((m,1)))
    minError = inf 
    for i in range(n):
        rangeMin = dataMatrix[:,i].min()
        rangeMax = dataMatrix[:,i].max()
        stepSize = (rangeMax-rangeMin)/numSteps
        for j in range(-1,int(numSteps)+1):
            for inequal in ['lt','gt']:
                threshVal = (rangeMin+ float(j)*stepSize)
                predictedVals = stumpClassify(dataMatrix,i,threshVal,inequal)
                errArr = mat(ones((m,1)))
                errArr[predictedVals==labelMat]==0
                weightedError = D.T*errArr
                print('dim %d,thresh %.2f thresh inequal %s,the weighted error is %.3f'%(i,threshVal,inequal,weightedError))
                if weightedError < minError:
                    minError = weightedError
                    bestClassEst = predictedVals.copy()
                    bestStump['dim']=i
                    bestStump['thresh']= threshVal
                    bestStump['ineq']=inequal
    return bestStump,minError,bestClassEst 
                

In [6]:
dataMat,classLabels = loadSimpData()
D = mat(ones((5,1))/5)
bestStump, minError, bestClasEst = buildStump(dataMat,classLabels,D)
print("最佳单层决策树相关信息：",bestStump)
print("最小错误率: ",minError)
print("预测的类别结果:",bestClasEst.T)

dim 0,thresh 0.90 thresh inequal lt,the weighted error is 1.000
dim 0,thresh 0.90 thresh inequal gt,the weighted error is 1.000
dim 0,thresh 1.00 thresh inequal lt,the weighted error is 1.000
dim 0,thresh 1.00 thresh inequal gt,the weighted error is 1.000
dim 0,thresh 1.10 thresh inequal lt,the weighted error is 1.000
dim 0,thresh 1.10 thresh inequal gt,the weighted error is 1.000
dim 0,thresh 1.20 thresh inequal lt,the weighted error is 1.000
dim 0,thresh 1.20 thresh inequal gt,the weighted error is 1.000
dim 0,thresh 1.30 thresh inequal lt,the weighted error is 1.000
dim 0,thresh 1.30 thresh inequal gt,the weighted error is 1.000
dim 0,thresh 1.40 thresh inequal lt,the weighted error is 1.000
dim 0,thresh 1.40 thresh inequal gt,the weighted error is 1.000
dim 0,thresh 1.50 thresh inequal lt,the weighted error is 1.000
dim 0,thresh 1.50 thresh inequal gt,the weighted error is 1.000
dim 0,thresh 1.60 thresh inequal lt,the weighted error is 1.000
dim 0,thresh 1.60 thresh inequal gt,the 

In [ ]:
def adaBoostTrainDS(dataArr,classLabels,numIt=40):
    weakClassArr = []
    m = shape(dataArr)[0]
    D = mat(ones(m,1))
    aggClassEst = mat(zeros((m,1)))
    for i in range(numIt):
        bestStump,error,classEst = buildStump(dataArr,classLabels,D)
        print('D:',D.T)
        alpha = float(0.5*log((1.0-error)/max(error,1e-16)))
        bestStump['alpha']=alpha
        weakClassArr.append(bestStump)
        print('classEst: ',classEst.T)
        expon = multiply(-1*alpha*mat(classLabels).T,classEst)
        D = multiply(D,exp(expon))
        D = D/D.sum()
        aggClassEst += aplpa*classEst
        print('aggClasssEst:',aggClassEst.T)
        aggErrors = multiply(sign(aggClassEst)!=mat(classLabels).T,)
        errorRate = aggErrors.sum()/m
        print('total error:',errorRate)
        if errorRate==0.0:
            break
    return weakClassArr

In [7]:
def adaClassify(dataToClass,classifierArr):
    dataMatrix = mat(dataToClass)
    m = shape(dataMatrix)[0]
    aggClassEst = mat(zeros((m,1)))
    for i in range(len(classifierArr)):
        classEst = stumpClassify(dataMatrix,classifierArr[i]['dim'],classifierArr[i]['thresh'],classifierArr[i]['ineq'])
        aggClassEst += classifierArr[i]['alpha']*classEst
        print(aggClassEst)
    return sign(aggClassEst)

In [8]:
def loadDataSet(filename):
    numFeat = len(open(filename).readline().split(' '))
    dataMat = []
    labelMat = []
    fr = open(filename)
    for line in fr.readlines():
        lineArr = []
        curLine = line.strip().split(' ')
        for i in range(numFeat-1):
            lineArr.append(float(curLine[i]))
        dataMat.append(lineArr)
        if int(curLine[-1])==1:
            labelMat.append(1.0)
        else:
            labelMat.append(-1.0)
    return dataMat,labelMat
